# Learning and Decision Making

## Laboratory 2: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the gridworld domain described in the Homework and which you modeled using a Markov decision process.

<img src="maze.png" width="200px">

Recall that:

* At each step, the agent may move in any of the four directions -- up, down, left and right. 
* Movement across a _grey_ cell division succeeds with a $0.8$ probability and fails with a $0.2$ probability. 
* Movements across colored cell divisions (blue or red) succeed with a $0.8$ probability _but only if the agent has the corresponding colored key_. Otherwise, they fail with probability $1$. 
* When the movement fails, the agent remains in the same cell. 
* To get a colored key, the agent simply needs to stand in the corresponding cell. 
* The goal of the agent is to reach the cell marked with **"G"**. 

**Throughout the lab, use $\gamma=0.99$.**

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs. Make sure that:
    * The costs lie in the interval [0, 1]
    * The cost for standing in goal cell is minimal
    * The cost for standing in intermediate cells is maximal

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [1]:
import numpy as np

#States
X = ('B0', 'C0', 'E0', 'F0', 'B1', 'C1', 'D1', 'E1', 'F1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2')
print("States: {}\n".format(X))

#Actions
A = ('U', 'D', 'L', 'R')
print("Actions: {}\n".format(A))

#Outcomes
#Action R(ight)
PR = np.zeros((len(X), (len(X))))
PR[0, 0] = 0.2
PR[0, 2] = 0.8
PR[1, 1] = 0.2
PR[1, 3] = 0.8
PR[2, 2] = 1
PR[3, 3] = 1
PR[4, 4] = 0.2
PR[4, 7] = 0.8
PR[5, 5] = 0.2
PR[5, 8] = 0.8
PR[6, 6] = 1
PR[7, 7] = 1
PR[8, 8] = 1
PR[9, 9] = 0.2
PR[9, 10] = 0.8
PR[10, 10] = 0.2
PR[10, 13] = 0.8
PR[11, 11] = 0.2
PR[11, 14] = 0.8
PR[12, 12] = 1
PR[13, 13] = 1
PR[14, 14] = 0.2
PR[14, 15] = 0.8
PR[15, 15] = 1
print("Transition Probability Matrix for Right action: \n{}\n".format(PR))

#Cost
C = np.zeros((len(X), len(A)))
C[:15] = 1
C[14, 3] = 0.2 # The agent may fail with 0.2 probability, paying a cost of 1
C[15, 0:2] = 0
C[15, 3] = 0
C[15, 2] = 1
C = np.round(C / np.max(C), 3) #Making sure all the costs are between 0 and 1, in case a weight is changed acidentaly
print("Cost Function: \n{}\n".format(C))

States: ('B0', 'C0', 'E0', 'F0', 'B1', 'C1', 'D1', 'E1', 'F1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2')

Actions: ('U', 'D', 'L', 'R')

Transition Probability Matrix for Right action: 
[[0.2 0.  0.8 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.2 0.  0.8 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.2 0.  0.  0.8 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.2 0.  0.  0.8 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.8 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.8 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.8 0. ]
 [0.  0.  

### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $x$, always moves the agent to the cell closest to the goal (irrespectively of the number of keys in the agent's possession). If multiple such cells exist, the agent should select randomly between the two.

For example, suppose that the agent is in cell 2. It should then select randomly between the actions $D$ and $R$. Conversely, suppose that the agent is in cell 4. The knight should then select actions $R$ with probability 1.

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry $(x,a)$ has the probability of selecting action $a$ in state $x$.

---

In [2]:
# -- Insert your code here -- #

---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2.

---

In [3]:
# -- Insert your code here -- #

### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is _not_ optimal: use value iteration to compute $J^*$ and show that $J^*\neq J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [4]:
# -- Insert your code here -- #

---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [5]:
# -- Insert your code here -- #

### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Suppose that the agent is where depicted in Fig. 1, and consider the situations (i) where it has no keys; (ii) where it has only the red key; (iii) where it has both keys. For each of the three situations,  

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [6]:
# -- Insert your code here -- #